In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import re

# Démarrage du chronomètre pour calculer la durée d'exécution du script
start_time = time.time()

# Importation du fichier CSV contenant les id
df_id = pd.read_csv('Data/df_for_scrap_imdb_filtre4.csv')

# Définition de la plage d'id à scraper
id_in = 0
id_out = len(df_id)
id = id_in

# Définition du nombre de requêtes effectuées en parallèle (Dépend du CPU)
nombre_req = 12

# Définition du chemin du fichier csv contenant les id
path_out = f'Data/imdb_titres_originaux_{id_in}_{id_out}.csv'

# Définition de la variable initial pour pour suivre la progression
pourcent = 0

# Définition du HEADERS pour que les requetes ne soit pas bloquées
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}


# Définition de la fonction pour scraper les données de IMDB
def imdb_titre_scrap(i):
    global pourcent, id#, time_limit
    id += 1
    titre = budget = titre_original = None
    
    try:
        # Construction de l'URL pour chaque film basé sur son ID
        url = f'https://www.imdb.com/title/{i}'
        res = session.get(url, allow_redirects=True, headers=HEADERS)
        # Print du % avec une intervalle de 72 tests
        pourcent += 1
        if pourcent == 10:
            print('Execution :', round(((id-id_in)/(id_out-id_in))*100, 2), '%')
            pourcent = 0
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if res is None or res.status_code != 200:
            return i, "0", f"Code HTTP = {res.status_code}"
        
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(res.content, 'lxml')
        #print(soup)
    
        titre_original = soup.find('div', class_=lambda x: x and 'sc-d8941411-1' in x)
        if titre_original:
            titre = titre_original.get_text()
            titre = titre.replace('Original title: ', '')
        else:
            titre_temp = soup.find('title')
            titre = titre_temp.get_text() if titre_temp else ''
            titre = titre.split(' (')[0].strip()
        
        budget_filtre1 = soup.find('div', {'data-testid': 'title-boxoffice-section'})
        if budget_filtre1 is not None:
            budget_balise = budget_filtre1.find('span', class_='ipc-metadata-list-item__list-content-item')
            if budget_balise is not None:
                budget = budget_balise.text.strip()


    # Gestion des exceptions liées aux requêtes HTTP
    except requests.exceptions.RequestException:
        return i, "0", "Exeption de la requête"
    print (i,' ',titre,' ',budget)
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    resultat = (
        i,
        titre if titre else "0",
        budget if budget else "0",
    )
    return resultat

# Initialisation d'un DataFrame pour stocker les données extraites   
df = pd.DataFrame(columns=['id', 'titre', 'budget']) 
session = requests.Session()

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=nombre_req) as executor:

    # Soumission des tâches de scraping pour chaque ID de film entre les valeurs id_in et id_out
    tests = [executor.submit(imdb_titre_scrap, i) for i in df_id['movie_id'][id_in:id_out]]
    for test in tests:
        i, titre, budget = test.result()
        if titre != "0":
            df.loc[i] = [i, titre, budget]
        else:
            print(i,' ', titre, ' ', budget, 'Impossible à enregistrer')

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv(path_out, index=False)

# Affichage du DataFrame final
display(df)